# Map interpolation

Interpolation is a kind of regression (and, arguably, all prediction is interpolation... so all machine learning models are maps). And we have a bunch of different nonlinear regressors available to us in `scikit-learn`. So let's apply some of them to a mapping problem.

We're going to start with a Gaussian process to model a surface. This is equivalent to kriging in the language of geostatistics. `sklearn` implements a sophisticated model, with stationary or non-stationary and isotropic or nonisotropic kernels — [read about it](https://scikit-learn.org/stable/modules/gaussian_process.html). 

[This tutorial](https://towardsdatascience.com/quick-start-to-gaussian-process-regression-36d838810319) is worth reading.

I'm using a small dataset originally from Geoff Bohling at the Kansas Geological Survey. I can no longer find the data online. We will look at making a map using the Scikit-Learn machine learning library with `sklearn.gaussian_process`.

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

## Read the data

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
<h3>EXERCISE</h3>

The data file is `../data/ZoneA.dat` (it's also on the web at `https://www.dropbox.com/s/6dyfc4fl5slhgry/ZoneA.dat?raw=1`)

Load the data from the text file into a Pandas DataFrame. <a title="Use Pandas's read_csv() method. You will need to set some arguments: sep, header, usecols, names, and dtype."><b>Hover for HINT</b></a>

We don't need all of the fields, but we at least need x, y, thickness and porosity. The top of the DataFrame should look something like this:

              x       y     thick       por
    0   12100.0  8300.0   37.1531   14.6515
    1    5300.0  8700.0   31.4993   14.5093
    
Once you have the data, can you make a KDE plot (`sns.kdeplot()`) with Seaborn?
</div>

In [ ]:
# YOUR CODE HERE
fname = "../data/ZoneA.dat"



This next bit looks a bit unpleasant, but we're just getting out min and max values for the x and y columns.

In [ ]:
extent = x_min, x_max, y_min, y_max = [df.x.min()-1000, df.x.max()+1000,
                                       df.y.min()-1000, df.y.max()+1000]

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
<h3>EXERCISE</h3>

Make a map of the porosity data. Use the colour of the points to indicate porosity, and use the size to indicate thickness of the zone.

Add a colourbar, title, and other annotation.
</div>

In [ ]:
# YOUR CODE HERE



## Make a grid

Now we can make a regular grid:

In [ ]:
# Notice the order of x (columns) and y (rows)!
grid_y, grid_x = np.mgrid[y_min:y_max:500, x_min:x_max:500]

# Use *shape* argument to specify the *number* (not size) of bins:
# grid_y, grid_x = np.mgrid[y_min:y_max:100j, x_min:x_max:100j]

plt.figure(figsize=(10, 6))
plt.scatter(grid_x, grid_y, marker='+', lw=0.5, s=10)

## Fit a model

In [ ]:
from sklearn.gaussian_process.kernels import RBF

kernel = RBF(length_scale=1000)

In [ ]:
from sklearn.gaussian_process import GaussianProcessRegressor

gp = GaussianProcessRegressor(normalize_y=True,
                              alpha=0.5,  # Larger values imply more noise in the input data.
                              kernel=kernel,)

gp.fit(df[['x', 'y']].values, df.por.values)

To make a prediction, we need to construct the `X` matrix: (x, y) coordinates in 2 columns:

In [ ]:
X_grid = np.stack([grid_x.ravel(), grid_y.ravel()]).T

Now we can make a prediction:

In [ ]:
y_grid = gp.predict(X_grid).reshape(grid_x.shape)

And plot the predicted grid with the input data using the same colourmap:

In [ ]:
# Compute min and max of all the data:
mi = np.min(np.hstack([y_grid.ravel(), df.por.values]))
ma = np.max(np.hstack([y_grid.ravel(), df.por.values]))

# Plot it all.
plt.figure(figsize=(15, 15))
im = plt.imshow(y_grid, origin='lower', extent=extent, vmin=mi, vmax=ma)
pts = plt.scatter(df.x, df.y, c=df.por, s=80, edgecolor='#ffffff66', vmin=mi, vmax=ma)
plt.colorbar(im, shrink=0.67)
plt.show()

## Evaluate the model

We can compute the residual error distribution by making a prediction on the original (x, y) values and comparing to the actual measured porosities at those locations:

In [ ]:
por_hat = gp.predict(df[['x', 'y']].values)

sns.kdeplot(por_hat - df.por)

We can also compute the RMS error:

In [ ]:
from sklearn.metrics import mean_squared_error

rmse = np.sqrt(mean_squared_error(df.por, por_hat))

print(f"RMS error: {rmse:.3f}")

## Going further

The GP model actually models the distribution, not just the mean (which for a normal distribution is also the most likely or mode, and the P50 or median).

So we can also look at samples from the distribution:

In [ ]:
y_sample = gp.sample_y(X_grid, n_samples=3, random_state=42)

fig, axs = plt.subplots(ncols=3, figsize=(15, 5))
for ax, y_hat in zip(axs, y_sample.T):
    im = ax.imshow(y_hat.reshape(grid_x.shape), origin='lower', extent=extent)
plt.show()

Let's look at a cross-section too.

We'll want the 'wells' on there at some point, so let's add grid-cell coordinates. (A better way to do this would be to use `xarray`.)

In [ ]:
df['x_grid'] = (df.x - x_min) // 500
df['y_grid'] = (df.y - y_min) // 500

Now we can get the mean and stdev of the prediction:

In [ ]:
y_grid, y_std = gp.predict(X_grid, return_std=True)

y_grid = y_grid.reshape(grid_x.shape)
y_std = y_std.reshape(grid_x.shape)

Then choose a row to look at, and filter the wells down to near that row:

In [ ]:
row = 19
wells = df[abs(df.y_grid - row) <= 1]

Let's check where that row and its wells are:

In [ ]:
plt.figure(figsize=(15, 10))
im = plt.imshow(y_grid, origin='lower', extent=extent, vmin=mi, vmax=ma)
pts = plt.scatter(df.x, df.y, c=df.por, s=80, edgecolor='#ffffff66', vmin=mi, vmax=ma)
pts = plt.scatter(wells.x, wells.y, c=wells.por, s=80, edgecolor='r', vmin=mi, vmax=ma)
plt.colorbar(im, shrink=0.67)
plt.axhline(y_min + 19 * 500, c='red')
plt.show()

And compute the various things:

In [ ]:
# Confidence interval.
phi, s = y_grid[row], y_std[row]
max_ = phi - 1.96 * s  # 95% confidence interval
min_ = phi + 1.96 * s  #     = 1.96 * stdev

# Draw some samples.
y_samples = gp.sample_y(X_grid, n_samples=5, random_state=42)

# Plot.
plt.figure(figsize=(15, 5))
for i, well in wells.iterrows():
    plt.scatter(well.x_grid, well.por, c='none', ec='r', s=80, zorder=100)
    plt.axvline(well.x_grid, c='k', alpha=0.15)
plt.plot(phi, c='darkblue', lw=4, alpha=0.67)
for sample in y_samples.T:
    plt.plot(sample.reshape(grid_x.shape)[row])
plt.fill_between(range(phi.size), min_, max_, alpha=0.2)
plt.show()

## More models

<div style="background: #e0ffe0; border: solid 2px #d0f0d0; border-radius:3px; padding: 1em; color: darkgreen">
<h3>EXERCISE</h3>

Try some other regressors in `scikit-learn`. Look at the maps they produce, and check their residuals. (You might want to write a loop to check several models.)

Which regressors produce the best maps? 
</div>

In [ ]:
# YOUR CODE HERE



If you're interested in this sort of thing, check out

- [This blog post](http://geologyandpython.com/ml-interpolation-method.html) on the subject.
- And [this paper](https://arxiv.org/pdf/2006.10461.pdf) on a clever CNN-based approach that incorporates Moran's I embedding.

## Superpowers with `verde` and `gstools`

The geophysical package `verde` contains still more gridding algorithms, and makes some aspects of the gridding much easier — for example reducing the nuber of points in the dataset in sensible ways. [Read about it.](https://www.leouieda.com/blog/introducing-verde.html) Better yet, [try it out.](https://github.com/agile-geoscience/xlines/blob/master/notebooks/11_Gridding_map_data.ipynb)

The geostatistical package `gstools` implements various kriging algorithms, which some people may be more familiar with. [Check it out](https://github.com/GeoStat-Framework/GSTools). 

----

<center>&copy; 2020 Agile Scientific, licensed CC-BY</center>